In [6]:
from glob import glob
import os
import json
import numpy as np
import pandas as pd
import argparse
import yaml
import torch
import random
import time
from typing import Optional

from tqdm.auto import tqdm
from tqdm import notebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import transformers
from transformers import BertConfig, BertModel, BertTokenizer

#check torch version & device
print ("PyTorch version:[%s]."%(torch.__version__))
print ("transformers version:[%s]."%(transformers.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.10.1+cu102].
transformers version:[4.8.1].
device:[cuda:0].


In [3]:
cfg = {
    'datadir' : '../data/Part1',
    'savedir' : '../data-styleT',
    'model'   : 'KRBERT/pytorch_model_char16424_ranked.bin',
    'config'  : 'KRBERT/bert_config.json',
    'MODEL'   : {'max_seq_length' : 512},
    'DATASET' : {'num_train_nonbait' : 20000,},
    'SEED':42    
    }

# DataLoader
- row로 일단 계산하고
- 두번째 시도 : matrix로 계산하기

In [7]:
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

tokenizer = BertTokenizer.from_pretrained('KRBERT/vocab.txt', do_lower_case=False)
config = BertConfig(cfg["config"])
model = BertForMaskedLM.from_pretrained(cfg['model'], config=cfg['config'])

/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1628: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at KRBERT/pytorch_model_char16424_ranked.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# 데이터 불러오기
nonbait_filelist = glob(os.path.join(cfg['datadir'], '[!sample]*/Clickbait_Auto/*/*'))
train_size = cfg['DATASET']['num_train_nonbait']
inference_size = len(nonbait_filelist) - train_size
nonbait_train, nonbait_infer = random_split(dataset = nonbait_filelist, 
                                            lengths = [train_size, inference_size], 
                                            generator = torch.Generator().manual_seed(42)
                                            )
nonbait_train_list = [nonbait_filelist[i] for i in nonbait_train.indices]
nonbait_infer_list = [nonbait_filelist[i] for i in nonbait_infer.indices]

bait_filelist = glob(os.path.join(cfg['datadir'], '[!sample]*/Clickbait_Direct/*/*'))
file_list = nonbait_train_list + bait_filelist

In [14]:
len(file_list)

70131

In [14]:
# 2. 데이터 불러오기
class PaddedDataset(Dataset):
    def __init__(self, file_list, tokenizer, max_seq_length, PAD = False):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.file_list = file_list
        self.PAD = PAD
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        input = self._get_text(self.file_list[idx])
        source = self.tokenizer(input, max_length = self.max_seq_length, 
                                padding = "max_length", truncation = True, 
                                )
        source_ids, target_ids = self.mask_token(source['input_ids'])

        if 'Clickbait_Direct' in self.file_list[idx]:
            token_type_ids = torch.tensor([1] * self.max_seq_length, dtype = torch.long)
        else:
            token_type_ids = torch.tensor([0] * self.max_seq_length, dtype = torch.long)

        return {
            "input_ids" : torch.tensor(source_ids, dtype=torch.long), 
            "attention_mask" : torch.tensor(source['attention_mask'], dtype=torch.long), 
            "token_type_ids" : token_type_ids,
            "labels" : torch.tensor(target_ids, dtype = torch.long)
            }

    def _get_text(self, file_path):
        source_file = json.load(open(file_path, "r"))
        title = source_file['sourceDataInfo']['newsTitle']
        content = source_file['sourceDataInfo']['newsContent']
        input_text = title + '[SEP]' + content
        return input_text
    
    def mask_token(self, input_ids : list, n = 4): 
        """
        BertTokenizer [SEP]를 사용한다고 가정
        input : w1, ..., wi-1, [MASK][MASK][MASK][MASK], wi+3, 
        label : 
        """
        label = input_ids.copy()

        # 1. title 부분에 [MASK]처리하기
        input_ids = np.array(input_ids)
        content_idx = np.where(input_ids == self.tokenizer.sep_token_id)[0]

        if self.PAD == False:
            rand_idx = random.randint(1,content_idx[0]-n) #[CLS] w1, w2, ..., wk, [SEP]에서 [SEP]이 겹치지 않게 mask하기

            ## input [MASK]처리하기
            input_ids[rand_idx : rand_idx+n] = self.tokenizer.mask_token_id
            label = np.array(label)
            label[:rand_idx] = -100  # We only compute loss on masked tokens
            label[rand_idx+n:] = -100

        elif self.PAD == True :
            ## 실제 mask 토큰의 개수(k) 구하기(1~4)
            n_masked = random.randint(1, n)
            rand_idx = random.randint(1,content_idx[0]-n_masked) #[CLS] w1, w2, ..., wk, [SEP]에서 [SEP]이 겹치지 않게 mask하기

            ## input [MASK]처리하기
            input_ids[rand_idx : rand_idx+n_masked] = self.tokenizer.mask_token_id

            ## pad token추가 되는 부분까지 [MASK]추가하기
            if n_masked != n :
                input_ids = np.hstack((input_ids[:rand_idx+n_masked], np.full(n-n_masked, self.tokenizer.mask_token_id), 
                                    input_ids[rand_idx+n_masked:]))
            ## label에 [PAD] 추가하기   
            label = np.hstack((label[:rand_idx+n_masked], np.full(n-n_masked, self.tokenizer.pad_token_id),
                            label[rand_idx+n_masked:],))

            # 2. loss계산 안 할 부분 찾기 : special token(cls, sep) + content
            label[:rand_idx] = -100  # We only compute loss on masked tokens
            label[rand_idx+n:] = -100

            ## maxlen 맞추기
            input_ids = np.hstack((input_ids[:self.max_seq_length-1], [tokenizer.sep_token_id]))
            label = label[:self.max_seq_length]

        return input_ids, label

In [15]:
train_size = int(0.8 * len(file_list))
test_size = len(file_list) - train_size

train_idx, test_idx = random_split(file_list, [train_size, test_size], generator=torch.Generator().manual_seed(42))

train_list=[file_list[i] for i in train_idx.indices]
test_list=[file_list[i] for i in test_idx.indices]

trainset = PaddedDataset(train_list, tokenizer, max_seq_length=512)
testset = PaddedDataset(test_list, tokenizer, max_seq_length=512)

# padded MLM

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset
)

In [12]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 80209
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100270
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


In [ ]:
trainer.save_model('./model_output')

NameError: name 'trainer' is not defined

# Making Token in source text

In [8]:
path = os.path.join(cfg['savedir'], 'infer.txt')
file_list_infer = open(path, "r").read().split("\n")
len(file_list_infer)

112519

In [13]:
model = BertForMaskedLM.from_pretrained("model_output/checkpoint-66000/")

In [14]:
def get_text(file_path):
    source_file = json.load(open(file_path, "r"))
    title = source_file['sourceDataInfo']['newsTitle']
    content = source_file['sourceDataInfo']['newsContent']
    text = title + '[SEP]' + content
    return text

In [39]:
def get_logit(file_path, cfg, model, span_size, SOURCE = None):
    text = get_text(file_path)
    input = tokenizer(text, 
                      max_length = cfg["MODEL"]["max_seq_length"], 
                      padding = "max_length", 
                      truncation = True, 
                      return_tensors="pt"
                      )
    if SOURCE :
        token_type_ids = torch.tensor([0] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #source
    else:
        token_type_ids = torch.tensor([1] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #target
    input["token_type_ids"] = token_type_ids
    with torch.no_grad():
        output = model(**input).logits
    indices = input.input_ids.unsqueeze(axis=-1) #(1, 512, 1)
    logit_of_input_ids = torch.gather(output, 2, indices).squeeze() #(1, 512, 1) : torch.gather 좋네

    ## input에서 [sep]의 index찾기
    source_sep_id = (input.input_ids[0] == tokenizer.sep_token_id).nonzero().squeeze()[0] 
    ## [sep]나오기 전까지 span길이만큼의 logit 합으로 구성된 matrix구하기
    n_gram_logits = torch.tensor([sum(logit_of_input_ids[i : i+span_size]) for i in range(0, source_sep_id - span_size + 1)])
    return n_gram_logits

In [61]:
import copy

def run(file_path, cfg, model, span_size):
    s_n_gram_logits = get_logit(file_path, cfg, model, span_size=4, SOURCE = True)
    t_n_gram_logits = get_logit(file_path, cfg, model, span_size=4, SOURCE = False)
    
    # span의 logit 차이가 큰 index부분 찾기 -> MASK할 부분
    diff = s_n_gram_logits-t_n_gram_logits
    mask_idx = diff.argmax() #source index로 사용하면 됨.

    text = get_text(file_path) 
    label = tokenizer(text, 
                      max_length = cfg["MODEL"]["max_seq_length"], 
                      padding = "max_length", 
                      truncation = True, 
                      return_tensors = "pt"
                      )
    masked_input = copy.deepcopy(label)
    masked_input['input_ids'][0, mask_idx : mask_idx+span_size] = tokenizer.mask_token_id
    masked_input['token_type_ids'] = torch.tensor([1] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #target
    
    return masked_input, label

In [62]:
masked_input, label = run(file_list_infer[0], cfg, model, span_size=4)

In [70]:
def save(input_list, label_list, savedir): 
    input_dict = {}
    for i, input in enumerate(tqdm(input_list)):
        if len(input_dict) == 0:
            for k in input.keys():
                input_dict[k] = []
        
        for k in input.keys():
            input_dict[k].append(input[k])

    for k in input_dict.keys():
        input_dict[k] = torch.cat(input_dict[k])
    label_list = torch.cat(label_list)

    torch.save({'input':input_dict, 'label':label_list}, os.path.join(savedir,'infer.pt'))

In [75]:
input_list = []
label_list = []
for i, file_path in enumerate(file_list_infer[:2]):
    masked_input, label = run(file_path, cfg, model, span_size=4)
    input_list.append(masked_input)
    label_list.append(label.input_ids)

save(input_list, label_list, cfg["savedir"])

100%|██████████| 2/2 [00:00<00:00, 9177.91it/s]


In [ ]:
# generate
data = torch.load(os.path.join(cfg["savedir"], 'infer.pt'))
input = {}
for k in data['input'].keys():
    input[k] = data['input'][k][i]
label = data['label'][i]

In [63]:
tokenizer.decode(masked_input['input_ids'][0])

'[CLS] 대전 갈마동에 갤 [MASK] [MASK] [MASK] [MASK]움 ‘ 분양 ’ [SEP] [UNK] 다우주택건설이 대전 갈마동에 공급하는 301세대 갤러리휴리움 조감도. 다우주택건설 ( 주 ), 7일 모델하우스 개장 … 301세대 분양 학군 · 지하철 역세권 혜택 … 10년만의 대규모 공급 대전 서구 갈마동에 27층 높이 301세대 규모의 신규 아파트단지가 들어선다. 더욱이 갈마동 지역에 10년 만에 나온 아파트 공급으로써 초ㆍ중ㆍ고 등 완성된 학군과 지하철역세권 혜택을 누릴 것으로 전망되면서 벌써부터 관심을 모으고 있다. 다우주택건설 ( 주 ) 이 오는 7일 갈마동 ‘ 갤러리휴리움 ’ 모델하우스를 개장하고 본격적인 분양에 돌입한다. 전용면적 51㎡ 126세대, 57㎡ 62세대, 59㎡ 55세대, 65㎡ 28세대로 구성된 중소형 평형의 갤러리휴리움은 최고 27층 높이다. 갈마1동주민센터와 옛 백년예식장 맞은 편에 위치한 서구 갈마동 315번지 일원에 건설되는 갤러리휴리움은 생활여건이 완성된 지역에서 10년 만에 이뤄지는 아파트 공급이라는 점에서 관심을 끈다. 인근 지역에서 최근 분양된 규모 있는 아파트가 없고, 10년 만에 선보이는 300세대급 공급인 셈이다. 봉산초ㆍ갈마중ㆍ한밭고 등 주변에 학군이 형성돼 있고 월평역과 갈마역까지 걸어서 접근할 수 있는 역세권에 해당한다. 또 한밭대로와 계룡로에 가까워 대부분 대전지역에서 쉽게 접근할 수 있는 교통망을 갖춘 상태다. 최고 27층 높이로 설계돼 갈마동 인근 건물 중 가장 높은 층을 확보하게 돼 눈에 쉽게 들어오는 랜드마크로 자리매김할 것으로 전망된다. 지역에 아파트를 공급하는 다우주택건설은 1999년 창립해 대전과 세종에서 갤러리빌이라는 브랜드의 주상복합 등을 13차례 성공 분양한 바 있다. 이번 갈마동 갤러리휴리움은 14차 공급으로써 편리한 시스템 구성으로 쾌적함과 안전함을 고루 갖춘 단지를 만들겠다는 포부다. 다우주택건설 전문수 회장은 [UNK] 삶의 가치를 담은 갤러리휴리움 아파트를 갈마동에

In [64]:
tokenizer.decode(label['input_ids'][0])

'[CLS] 대전 갈마동에 갤러리휴리움 ‘ 분양 ’ [SEP] [UNK] 다우주택건설이 대전 갈마동에 공급하는 301세대 갤러리휴리움 조감도. 다우주택건설 ( 주 ), 7일 모델하우스 개장 … 301세대 분양 학군 · 지하철 역세권 혜택 … 10년만의 대규모 공급 대전 서구 갈마동에 27층 높이 301세대 규모의 신규 아파트단지가 들어선다. 더욱이 갈마동 지역에 10년 만에 나온 아파트 공급으로써 초ㆍ중ㆍ고 등 완성된 학군과 지하철역세권 혜택을 누릴 것으로 전망되면서 벌써부터 관심을 모으고 있다. 다우주택건설 ( 주 ) 이 오는 7일 갈마동 ‘ 갤러리휴리움 ’ 모델하우스를 개장하고 본격적인 분양에 돌입한다. 전용면적 51㎡ 126세대, 57㎡ 62세대, 59㎡ 55세대, 65㎡ 28세대로 구성된 중소형 평형의 갤러리휴리움은 최고 27층 높이다. 갈마1동주민센터와 옛 백년예식장 맞은 편에 위치한 서구 갈마동 315번지 일원에 건설되는 갤러리휴리움은 생활여건이 완성된 지역에서 10년 만에 이뤄지는 아파트 공급이라는 점에서 관심을 끈다. 인근 지역에서 최근 분양된 규모 있는 아파트가 없고, 10년 만에 선보이는 300세대급 공급인 셈이다. 봉산초ㆍ갈마중ㆍ한밭고 등 주변에 학군이 형성돼 있고 월평역과 갈마역까지 걸어서 접근할 수 있는 역세권에 해당한다. 또 한밭대로와 계룡로에 가까워 대부분 대전지역에서 쉽게 접근할 수 있는 교통망을 갖춘 상태다. 최고 27층 높이로 설계돼 갈마동 인근 건물 중 가장 높은 층을 확보하게 돼 눈에 쉽게 들어오는 랜드마크로 자리매김할 것으로 전망된다. 지역에 아파트를 공급하는 다우주택건설은 1999년 창립해 대전과 세종에서 갤러리빌이라는 브랜드의 주상복합 등을 13차례 성공 분양한 바 있다. 이번 갈마동 갤러리휴리움은 14차 공급으로써 편리한 시스템 구성으로 쾌적함과 안전함을 고루 갖춘 단지를 만들겠다는 포부다. 다우주택건설 전문수 회장은 [UNK] 삶의 가치를 담은 갤러리휴리움 아파트를 갈마동에 선보여 최고이자 모범단지로서 랜드마크가 되

In [65]:
masked_input['token_type_ids']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# DRAFT 🚷

In [71]:
input = get_text(file_list_infer[0])
source = tokenizer(input, max_length = cfg["MODEL"]["max_seq_length"], padding = "max_length", truncation = True, return_tensors="pt")
target = tokenizer(input, max_length = cfg["MODEL"]["max_seq_length"], padding = "max_length", truncation = True, return_tensors="pt")

token_type_ids = torch.tensor([0] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #source
source["token_type_ids"] = token_type_ids

token_type_ids = torch.tensor([1] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #target
target["token_type_ids"] = token_type_ids

with torch.no_grad():
    s_outputs = model(**source).logits
    t_outputs = model(**target).logits

In [110]:
s_indices = source.input_ids.unsqueeze(axis=-1) #(1, 512, 1)
s_logit_of_input_ids = torch.gather(s_outputs, 2, s_indices).squeeze() #(1, 512, 1) : torch.gather 좋네

t_indices = target.input_ids.unsqueeze(axis=-1) #(1, 512, 1)
t_logit_of_input_ids = torch.gather(t_outputs, 2, s_indices).squeeze() #(1, 512, 1)

In [140]:
# span길이만큼의 logit 합으로 구성된 matrix구하기
span_size = 4
source_sep_id = (source.input_ids[0] == tokenizer.sep_token_id).nonzero().squeeze()[0]
s_n_gram_logits = torch.tensor([sum(s_logit_of_input_ids[i : i+span_size]) for i in range(0, source_sep_id - span_size + 1)])
t_n_gram_logits = torch.tensor([sum(t_logit_of_input_ids[i : i+span_size]) for i in range(0, source_sep_id - span_size + 1)])

In [146]:
# logit 차이가 가장 큰 span index 찾기
diff = s_n_gram_logits-t_n_gram_logits
mask_idx = diff.argmax() #source index로 사용하면 됨.
print(mask_idx)

# logit이 가장 큰 index에 masking하기
source.input_ids[0, mask_idx : mask_idx+span_size] = tokenizer.mask_token_id

tensor(13)


In [147]:
source.input_ids

tensor([[    2,  5050,  2342,     8,   516,  1889,     9,  1540,    28,    81,
           670, 10011,    11,     4,     4,     4,     4, 10011,    11,     3,
           440,   320,   113,  1284,    81,   219,    10,   159,   157,   607,
          5136, 12137,   107,   124,  5050,  5136,   516,  1889,     9,  1896,
            26,    30,     5,   741,  1612,   565,   159,   157,   607,  4208,
             9,  6076,  6426,   879,   833,    78,  2342,     7,   366,  1612,
          1087,   159,   157,   607,  5136,   301,    10,  1941,    87,    68,
           284,  1340,   159,    16,  6034,  1027,  3005,     5,   159,   157,
           607,  1540,  6426,  3555,  2817,    72,   266,  5285,  1153,   277,
          2597,  1505,  3773,   440,   320,   113,    10,   912,   359,    32,
          2270,  1427,  1540,     6,  1844,    13,  1043,  4974,     9,   658,
           464,   681,  1436,   369,   605,    16,  1219,  6068,    16,  6402,
          2232,  1599,    21,  1574,     5,  1540,  

In [149]:
tokenizer.decode(source.input_ids[0])

'[CLS] 식품업계, 줄줄이 가격인상 … \\ " [MASK] [MASK] [MASK] [MASK] \\ " [SEP] 인건비 급상승에 원재료 가격이 겹치며 식품 가격이 줄줄이 오르고 있다. 전체 매출 가운데 원재료 값이 차지하는 비중이 높은 라면업계의 경우 매출 대비 원재료 가격이 지난해에 비해 10 % 이상 올라 원가 부담이 크게 늘었다. 원재료 가격 비중이 다소 낮은 가공식품업체들은 매년 상승해온 인건비에 버티지 못하고 상품 가격을 올리는 악순환이 계속되고 있어 내년에도 물가 인상 기조가 이어질 것이라는 전망도 나온다. 가격 올려도 남는 게 없다24일 라면 업체들에 따르면 주요 제품 가격을 10 % 안팎으로 올렸지만 여전히 실적은 부진한 것으로 나타났다. 주 재료인 소맥과 팜유 가격이 급등하며 전체 매출 중 원재료 값이 차지하는 비중이 급증하고 있기 때문이다. 농심의 경우 지난해 전체 매출 가운데 원재료 값이 차지하는 비중이 43 % 인 것으로 나타났다. 2018년 35 % 에 불과했던 것에 비해 8 % 증가했다. 올해는 전체 매출 중 원재료 값이 차지하는 비중은 최소 50 % 를 넘어설 것으로 전망된다. 농심은 지난해 소맥을 t당 202달러에 수입했으나 올해에는 258달러로 27 % 비싼 가격에 들여왔다. 팜유의 경우 지난해 t당 627달러에서 올해 1110달러로 100 % 가까이 오르며 원가 부담이 급격히 높아졌다. 오뚜기, 삼양식품 등도 마찬가지인 상황이다. 오뚜기의 경우 지난해 전체 매출 가운데 원재료 값이 차지하는 비중은 69 % 였으나 올해 3분기까지 원재료 값이 차지하는 비중은 81 % 로 높아졌다. 삼양식품의 경우 원재료 값 비중이 지난해 56 % 에서 올해 60 % 이상을 기록할 것으로 보인다. 특히 삼양식품의 경우 전체 매출 가운데 라면이 차지하는 비중이 90 % 로 제품 가격 인상에도 영업이익은 하락할 것으로 전망된다. 너도 나도 ‘ 가격인상 ’ 원재료 값 상승에 영향을 덜 받은 식품업체들은 급등한 인건비가 문제다. CJ제일제당의 경

In [150]:
data_for_gen = source.copy()
data_for_gen["token_type_ids"] = torch.tensor([1] * cfg["MODEL"]["max_seq_length"], dtype = torch.long) #target

## Generation text

In [154]:
with torch.no_grad():
    outputs = model(**data_for_gen).logits

In [155]:
# retrieve index of [MASK]
mask_token_index = (data_for_gen.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = outputs[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id) #학습이 제대로 안됨... 나 어떠카지...

'내년도 [PAD] [PAD]'

In [159]:
tokenizer.decode(target.input_ids[0])

'[CLS] 식품업계, 줄줄이 가격인상 … \\ " 남는 게 없다 \\ " [SEP] 인건비 급상승에 원재료 가격이 겹치며 식품 가격이 줄줄이 오르고 있다. 전체 매출 가운데 원재료 값이 차지하는 비중이 높은 라면업계의 경우 매출 대비 원재료 가격이 지난해에 비해 10 % 이상 올라 원가 부담이 크게 늘었다. 원재료 가격 비중이 다소 낮은 가공식품업체들은 매년 상승해온 인건비에 버티지 못하고 상품 가격을 올리는 악순환이 계속되고 있어 내년에도 물가 인상 기조가 이어질 것이라는 전망도 나온다. 가격 올려도 남는 게 없다24일 라면 업체들에 따르면 주요 제품 가격을 10 % 안팎으로 올렸지만 여전히 실적은 부진한 것으로 나타났다. 주 재료인 소맥과 팜유 가격이 급등하며 전체 매출 중 원재료 값이 차지하는 비중이 급증하고 있기 때문이다. 농심의 경우 지난해 전체 매출 가운데 원재료 값이 차지하는 비중이 43 % 인 것으로 나타났다. 2018년 35 % 에 불과했던 것에 비해 8 % 증가했다. 올해는 전체 매출 중 원재료 값이 차지하는 비중은 최소 50 % 를 넘어설 것으로 전망된다. 농심은 지난해 소맥을 t당 202달러에 수입했으나 올해에는 258달러로 27 % 비싼 가격에 들여왔다. 팜유의 경우 지난해 t당 627달러에서 올해 1110달러로 100 % 가까이 오르며 원가 부담이 급격히 높아졌다. 오뚜기, 삼양식품 등도 마찬가지인 상황이다. 오뚜기의 경우 지난해 전체 매출 가운데 원재료 값이 차지하는 비중은 69 % 였으나 올해 3분기까지 원재료 값이 차지하는 비중은 81 % 로 높아졌다. 삼양식품의 경우 원재료 값 비중이 지난해 56 % 에서 올해 60 % 이상을 기록할 것으로 보인다. 특히 삼양식품의 경우 전체 매출 가운데 라면이 차지하는 비중이 90 % 로 제품 가격 인상에도 영업이익은 하락할 것으로 전망된다. 너도 나도 ‘ 가격인상 ’ 원재료 값 상승에 영향을 덜 받은 식품업체들은 급등한 인건비가 문제다. CJ제일제당의 경우 전체 매출 중 급여가 차지하는 비

In [161]:
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(target.input_ids[0] == tokenizer.mask_token_id, labels, -100)

outputs = model(**target, labels=labels)
round(outputs.loss.item(), 2)

RuntimeError: The size of tensor a (512) must match the size of tensor b (17) at non-singleton dimension 1